<a href="https://colab.research.google.com/github/davidwhogg/LearnDimensionalConstant/blob/main/notebooks/learn_a_dimensional_constant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning dimensional constants

In [ ]:
!pip install smithnormalform
from smithnormalform import snfproblem
from smithnormalform import matrix as snfmatrix
from smithnormalform import z as snfz
import numpy as np
import pylab as plt
import itertools as it

In [ ]:
rng = np.random.default_rng(17)

In [ ]:
# define units-carrying object
class quantity():
    unit_names = ["kg", "m", "s", "K"]

    def __init__(self, x, u):
        self.x = x
        self.u = np.array(u).astype(int)
        assert len(self.u) == len(self.unit_names)

    def unit_string(self):
        foo = ""
        for i, alpha in enumerate(self.u):
            if alpha != 0:
                foo += "\," + self.unit_names[i] + "^{" + str(alpha) + "}"
        return foo

    def __str__(self):
        return "$" + str(self.x) + self.unit_string() + "$"

    def __mul__(self, other):
        return quantity(self.x * other.x, self.u + other.u)

    def __truediv__(self, other):
        return quantity(self.x / other.x, self.u - other.u)

    def __pow__(self, b):
        return quantity(self.x ** b, self.u * b)

    def __add__(self, other):
        assert self.u == other.u
        return quantity(self.x + other.x, self.u)

    def __sub__(self, other):
        assert self.u == other.u
        return quantity(self.x - other.x, self.u)


In [ ]:
# define physical constants
# examples:
# temperature has units [0 0 0 1]
# wavelength has units [0 1 0 0]
c = quantity(299_792_458.0, [0, 1, -1, 0]) #speed of light
kB = quantity(1.380649e-23, [1, 2, -2, -1]) #Boltzmann's constant
h = quantity(6.62607015e-34, [1, 2, -1, 0]) #Planck's constant
print(c, kB, h)
print(h * c)
print(c / kB)

In [ ]:
# define God's truth (the true black body formula)
def truth(lam, T):
    return (quantity(2., [0, 0, 0, 0]) * h * c ** 2 / lam ** 5
            / quantity(np.exp((h * c / (lam * kB * T)).x) - 1., [0, 0, 0, 0]))

In [ ]:
def noisify(ys):
    return ys*(1+ 0.1* rng.normal(size=ys.shape))

In [ ]:
# set up parameters for a training set
lam = quantity(10. ** np.arange(-8, -4, 0.1), [0, 1, 0, 0]) #wavelengths

In [ ]:
# make a training set
# data and physical constants
Ts = [300., 1000., 3000., 10000., 30000.] #temperatures
Ts = [quantity(np.zeros_like(lam.x) + T, [0, 0, 0, 1]) for T in Ts]
B_lams = [truth(lam, T) for T in Ts] #intensities
ys = noisify(np.array([B.x for B in B_lams]).flatten())
y_u = B_lams[0].u
n = len(ys)
xs = np.zeros((n, 5)) #values
x_u = np.zeros((5, 4)).astype(int) #units
xs[:, 0] = np.array([lam.x for B in B_lams]).flatten()
x_u[0, :] = lam.u
xs[:, 1] = np.array([T.x for T in Ts]).flatten()
x_u[1, :] = Ts[0].u
xs[:, 2] = c.x
x_u[2, :] = c.u
xs[:, 3] = kB.x
x_u[3, :] = kB.u
xs[:, 4] = h.x
x_u[4, :] = h.u

In [ ]:
# make a test set
Ts_test = [6000., 20000., 60000.]
Ts_test = [quantity(np.zeros_like(lam.x) + T, [0, 0, 0, 1]) for T in Ts_test]
B_lams_test = [truth(lam, T) for T in Ts_test]
ys_test = noisify(np.array([B.x for B in B_lams_test]).flatten())
n_test = len(ys_test)
xs_test = np.zeros((n_test, 5))
xs_test[:, 0] = np.array([lam.x for B in B_lams_test]).flatten()
xs_test[:, 1] = np.array([T.x for T in Ts_test]).flatten()
xs_test[:, 2] = c.x
xs_test[:, 3] = kB.x
xs_test[:, 4] = h.x

In [ ]:
# cut on intensity <1e6
y_min = 1e6
i_train = ys > y_min
xs = xs[i_train]
ys = ys[i_train]
i_test = ys_test > y_min
xs_test = xs_test[i_test]
ys_test = ys_test[i_test]

In [ ]:
# what are the units of the labels and features?
print(y_u)
print(x_u)

In [ ]:
# visualize the training set
plt.scatter(xs[:, 0], ys, marker=".", c=xs[:, 1])
plt.colorbar()
plt.ylim(1.e6, 1.e18)
plt.ylabel("intensity " + truth(quantity(1.0, [0, 1, 0, 0]), Ts[0]).unit_string())
plt.xlabel("wavelength (m)")
plt.loglog()

Non-units-covariant regression using only wavelengths and temperatures as features

Note: **currently the baseline regression does not work**

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(hidden_layer_sizes=(20,20,20), learning_rate_init=1e-4, random_state=1, max_iter=20000).fit(np.log(xs[:,:2]), np.log(ys))

In [ ]:
ys_hat_baseline= np.exp(regr.predict(np.log(xs_test[:,:2])))

In [ ]:
# visualize the test results
plt.scatter(xs_test[:, 0], ys_test, marker=".", label="truth", c="black")
plt.scatter(xs_test[:, 0], ys_hat_baseline, marker=".", label="predictions", c="gray")
#plt.colorbar()
#plt.ylim(1.e6, 1.e20)
plt.ylabel(truth(quantity(1.0, [0, 1, 0, 0]), Ts[0]).unit_string())
plt.loglog()

rms_baseline = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat_baseline))**2))
print("rms_baseline", rms_baseline)

Units-covariant regression

In [ ]:
def integer_solve(A, b):
    """
    Find all solutions to Ax=b where A, x, b are integer.

    ## inputs:
    - A - [n, m] integer matrix - n < m, please
    - b - [n] integer vector

    ## outputs:
    - vv - [m] one integer vector solution to the problem Ax=b
    - us - [k, m] set of k integer vector solutions to the problem Ax=0

    ## bugs / issues:
    - Might get weird when k <= 1.
    - Might get weird if k > m - n.
    - Depends EXTREMELY strongly on everything being integer.
    - Uses smithnormalform package, which is poorly documented.
    - Requires smithnormalform package to have been imported as follows:
        !pip install smithnormalform
        from smithnormalform import snfproblem
        from smithnormalform import matrix as snfmatrix
        from smithnormalform import z as snfz
    """
    ## perform the packing into SNF Matrix format; HACK
    n, m = A.shape
    assert(m >= n)
    assert(len(b) == n)
    assert A.dtype is np.dtype(int)
    assert b.dtype is np.dtype(int)
    smat = snfmatrix.Matrix(n, m, [snfz.Z(int(a)) for a in A.flatten()])
    ## calculate the Smith Normal Form
    prob = snfproblem.SNFProblem(smat)
    prob.computeSNF()
    ## perform the unpacking from SNF Matrix form; HACK
    SS = np.array([a.a for a in prob.S.elements]).reshape(n, n)
    TT = np.array([a.a for a in prob.T.elements]).reshape(m, m)
    JJ = np.array([a.a for a in prob.J.elements]).reshape(n, m)
    ## Find a basis for the lattice of null vectors
    us = None
    zeros = np.sum(JJ ** 2, axis=0) == 0
    us = (TT[:, zeros]).T
    DD = SS @ b
    v = np.zeros(m)
    v[:n] = DD / np.diag(JJ)
    vv = (TT @ v).astype(int)
    return vv, us

In [ ]:
def create_dimensionless_features(vv, us, X, X_test):
    exponents = np.vstack([np.zeros_like(vv), us]).T
    features = np.exp(np.log(X)@exponents)
    features_test = np.exp(np.log(X_test)@exponents)
    return features, features_test

In [ ]:
def create_dimensionless_labels(vv, X, Y):
    return np.exp(np.log(Y) - np.log(X)@vv)


If we use Planck's constant and make the problem fully dimensionless, all curves in the training set collapse into one curve.

In [ ]:
vv, us = integer_solve(x_u.T, y_u)
XX, XX_test = create_dimensionless_features(vv, us, xs, xs_test)
YY = create_dimensionless_labels(vv, xs, ys)

plt.scatter(XX[:, 1], YY, marker=".")
plt.loglog()
plt.ylim(1e-14, 1e2)
plt.ylabel("dimensionless label")
plt.xlabel("dimensionless feature 1")

In [ ]:
def dimensionless_regression(X, Y, a_X, a_Y, X_test):
    vv, us = integer_solve(a_X.T, a_Y)
    XX, XX_test = create_dimensionless_features(vv, us, X, X_test)
    YY = create_dimensionless_labels(vv, X, Y)
    regr = MLPRegressor(hidden_layer_sizes=(20,20,20), learning_rate_init=1e-4, random_state=1, max_iter=20000).fit(np.log(XX), np.log(YY))
    dimensionless_predictions = regr.predict(np.log(XX_test))
    predictions = np.exp(dimensionless_predictions + np.log(X_test)@vv)
    return predictions

Perform a units-covariant regression without Planck's constant:

In [ ]:
vv, us = integer_solve(x_u[:-1,:].T, y_u) # :-1 removes Plack's constant from the features
print(vv)
print(us)
#the outputs show that there is no dimensionless features, but there is an output with the correct dimensions

In [ ]:
ys_hat_noplanck = dimensionless_regression(xs[:,:-1], ys, x_u[:-1,:], y_u, xs_test[:,:-1])
plt.scatter(xs_test[:, 0], ys_test, marker=".",  alpha=.75, label="truth")
plt.scatter(xs_test[:, 0], ys_hat_noplanck, marker=".", alpha=.75, label="predictions")
#plt.ylim(1.e6, 1.e17)
plt.loglog()
plt.legend()
plt.xlabel("wavelength")
plt.ylabel("intensity")
plt.title("This is pretty bad")

rms = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat_noplanck))**2))
print("rms", rms)

Perform a units-covariant regression with Planck's constant:

In [ ]:
vv, us = integer_solve(x_u.T, y_u)
print(vv)
print(us)

In [ ]:
ys_hat_planck = dimensionless_regression(xs, ys, x_u, y_u, xs_test)
plt.scatter(xs_test[:, 0], ys_test, marker=".", alpha=.75, label="truth")
plt.scatter(xs_test[:, 0], ys_hat_planck, marker=".", alpha=.75, label="predictions")
rms = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat_planck))**2))
print("rms", rms)
#plt.ylim(1.e6, 1.e17)
plt.loglog()
plt.legend()
plt.xlabel("wavelength")
plt.ylabel("intensity")
plt.title("This is pretty good")

In [ ]:
ys_hat_planck = dimensionless_regression(xs, ys, x_u, y_u, xs_test)
plt.scatter(xs_test[:, 0], ys_test, marker=".", alpha=1, label="test labels", c="black")
plt.scatter(xs_test[:, 0], ys_hat_noplanck, marker=".", alpha=.5, label="units equivariant without extra dimensional constant", c="gray")
plt.scatter(xs_test[:, 0], ys_hat_planck, marker="o", alpha=.5, label="units equivariant with extra dimensional constant", c="gray", Facecolors=None)
plt.scatter(xs_test[:, 0], ys_hat_baseline, marker="x", alpha=.5, label="standard MLP (no equivariance)", c="gray", Facecolors="green" )
rms = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat_planck))**2))
print("rms", rms)
#plt.ylim(1.e6, 1.e17)
plt.loglog()
plt.legend(loc='center', bbox_to_anchor=(0.5, 1.2))
plt.xlabel("wavelength")
plt.ylabel("intensity")
#plt.title("This is pretty good")
plt.savefig("units.pdf", bbox_inches='tight')

We try lots of dimensional constants in place of Planck's constant. Which ones work?

In [ ]:
#we are using brute force
#all possible units of the constants we are searching
try_us = list(it.product([-1,0,1], repeat=4))
print(try_us[:5])

In [ ]:
#finds every possible dimensional constant that leads to a good regression

for try_u in try_us:
    xs_aux = xs.copy()
    xs_test_aux = xs_test.copy()
    xs_aux[:,-1] = 1.0
    xs_test_aux[:,-1] = 1.0
    x_u_aux = x_u.copy()
    x_u_aux[-1] = try_u
    vv, us = integer_solve(x_u_aux.T, y_u)
    #print(vv)
    #print(us)
    assert len(us)>0
    XX_aux, XX_aux_test = create_dimensionless_features(vv, us, xs_aux, xs_test_aux)
    YY_aux = create_dimensionless_labels(vv, xs_aux, ys)
    foo = np.nanmedian(XX_aux[:,1])
    #finds the scaling to make the median of the dimensionless features unity
    factor = foo**(1.0 / us[0][-1])
    xs_aux[:,-1] = 1.0 / factor
    xs_test_aux[:,-1] = 1.0 / factor
    XX_aux, XX_aux_test = create_dimensionless_features(vv, us, xs_aux, xs_test_aux)
    YY_aux = create_dimensionless_labels(vv, xs_aux, ys)
    ys_hat = dimensionless_regression(xs_aux, ys, x_u_aux, y_u, xs_test_aux)
    rms = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat))**2))
    if rms<rms_baseline:
        print(xs_aux[0,-1], try_u, rms)
        break
    #assert False

In [ ]:
#check this out: we get Planck's constat to within a factor of 2
#it is this line: 3.5618249131350745e-34 (1, 2, -1, 0) 0.1753106257741744
#todo: interpret the results of the above

In [ ]:
ys_hat_planck = ys_hat
plt.scatter(xs_test[:, 0], ys_test, marker=".", alpha=1, label="test labels", c="black")
plt.scatter(xs_test[:, 0], ys_hat_noplanck, marker=".", alpha=.5, label="units covariant without extra dimensional constant, RMSE=6.35", c="gray")
plt.scatter(xs_test[:, 0], ys_hat_planck, marker="o", alpha=.5, label="units covariant with extra dimensional constant, RMSE=0.39", c="gray", Facecolors=None)
plt.scatter(xs_test[:, 0], ys_hat_baseline, marker="x", alpha=.5, label="standard MLP (no covariance), RMSE=0.64", c="gray", Facecolors="green" )
rms = np.sqrt(np.nanmean((np.log(ys_test) - np.log(ys_hat_planck))**2))
print("rms", rms)
#plt.ylim(1.e6, 1.e17)
plt.loglog()
plt.legend(loc='center', bbox_to_anchor=(0.5, 1.17))
plt.xlabel("wavelength")
plt.ylabel("intensity")
#plt.title("This is pretty good")
plt.savefig("units.pdf", bbox_inches='tight')